In [ ]:
import sentiment
import pandas as pd
import nltk
import pickle
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
print(sentiment.df.head())

In [ ]:
df = sentiment.df

In [ ]:
tfdif = TfidfVectorizer(stop_words='english')
df['content'] = df['content'].fillna('')
tfidf_matrix = tfdif.fit_transform(df['content'])

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
print(indices.head())

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    news_indices = [i[0] for i in sim_scores]
    print(sim_scores)
    return df['title'].iloc[news_indices]

In [ ]:
get_recommendations('India’s Modi wins election, but BJP suffers setback')


In [ ]:
df['tags'] = df['content'].copy()

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))

    return ' '.join(l)

In [ ]:
df['tags'] = df['tags'].apply(stems)

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vector = cv.fit_transform(df['tags']).toarray()

In [ ]:
similartiy = cosine_similarity(vector)

In [ ]:
similartiy.shape

In [ ]:
df[df['title'] == 'India election 2024: Why isn’t Modi’s BJP fielding candidates in Kashmir?'].index[0]

In [ ]:
def recommend(news):
    index = df[df['title'] == news].index[0]
    distances = sorted(list(enumerate(similartiy[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(df.iloc[i[0]].title)

In [ ]:
recommend('India’s Modi wins election, but BJP suffers setback')

In [ ]:
pickle.dump(df, open('news.pkl', 'wb'))
pickle.dump(similartiy, open('similarity.pkl', 'wb'))